In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/tweets.csv")
df.info()

# Preprocessing :


### 1. deleting duplicated tweets

In [ ]:
df.drop_duplicates(subset = 'Tweets', keep = 'first', inplace = True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106293 entries, 0 to 106292
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Tweet ID  106293 non-null  object
 1   Tweets    106293 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


### 2. Deleting retweets (this is in accrdance with the pre processing method followed in reference [1])

In [ ]:
retweets = df['Tweets'].str.startswith('RT')

In [ ]:
retweets.describe()

count     106293
unique         2
top        False
freq      106138
Name: Tweets, dtype: object

##### Thus we can see that one of the tweets is a retweet, hence we delete it from the dataset

In [ ]:
x = df[df['Tweets'].str.startswith('RT ')]
x

,Tweet ID,Tweets
284,1383347585386041088,RT @swachhbharat RT @MoHFW_INDIA: Always stand...
1384,1383479141639482880,RT for National Cause.\n\n#remdesivirinjection...
2028,1383352597961056000,RT @ndtvfeed: UP Pharmacist Sacked For Giving ...
2029,1383352591141129984,"RT @ndtvfeed: Over 1,700 Test COVID-19 Positiv..."
2030,1383352555338551040,RT @ndtvfeed: Akhilesh Yadav Tests Covid Posit...
...,...,...
102416,1390221744871415808,RT if you think complete #lockdown is the only...
103107,1390106648568639489,"RT @USAID: Within six days, six flights full o..."
103108,1390106647079669763,"RT @USAID: Within six days, six flights full o..."
105677,1390203016482107393,RT A poem inspired by the Khandav van dahan ep...


In [ ]:
#912, 92449  , 24756 48795   62442  66968  
print([*x.index])

[284, 1384, 2028, 2029, 2030, 5246, 9000, 9164, 9239, 9241, 11098, 12984, 12989, 14460, 14694, 15432, 15442, 15455, 15754, 15756, 17655, 17660, 17858, 19794, 19795, 19913, 20087, 25193, 32417, 32418, 34453, 34889, 36459, 36586, 37652, 37654, 37655, 38201, 38499, 39577, 40830, 41711, 41716, 43953, 44474, 45359, 46093, 47089, 47401, 49076, 49602, 49603, 50514, 50628, 52013, 52480, 54123, 56290, 56292, 56300, 57264, 58391, 59975, 59976, 59977, 60673, 61192, 61197, 61516, 61754, 62727, 63453, 65871, 66288, 66551, 66672, 66951, 69401, 69402, 70631, 71383, 72503, 72964, 73009, 73365, 73610, 73656, 76877, 78477, 79748, 79749, 80241, 80359, 85117, 85895, 85896, 86010, 86011, 86123, 86238, 86243, 87687, 88252, 88253, 89384, 89404, 89405, 91115, 91116, 91153, 91797, 91801, 91818, 92210, 92331, 92332, 93723, 96753, 97396, 97484, 98183, 98184, 98377, 98381, 98596, 99453, 99542, 99593, 101831, 102183, 102416, 103107, 103108, 105677, 106041]


In [ ]:
rem =[284, 1384, 2028, 2029, 2030, 5246, 9000, 9164, 9239, 9241, 11098, 12984, 12989, 14460, 14694, 15432, 15442, 15455, 15754, 15756, 17655, 17660, 17858, 19794, 19795, 19913, 20087, 25193, 32417, 32418, 34453, 34889, 36459, 36586, 37652, 37654, 37655, 38201, 38499, 39577, 40830, 41711, 41716, 43953, 44474, 45359, 46093, 47089, 47401, 49076, 49602, 49603, 50514, 50628, 52013, 52480, 54123, 56290, 56292, 56300, 57264, 58391, 59975, 59976, 59977, 60673, 61192, 61197, 61516, 61754, 62727, 63453, 65871, 66288, 66551, 66672, 66951, 69401, 69402, 70631, 71383, 72503, 72964, 73009, 73365, 73610, 73656, 76877, 78477, 79748, 79749, 80241, 80359, 85117, 85895, 85896, 86010, 86011, 86123, 86238, 86243, 87687, 88252, 88253, 89384, 89404, 89405, 91115, 91116, 91153, 91797, 91801, 91818, 92210, 92331, 92332, 93723, 96753, 97396, 97484, 98183, 98184, 98377, 98381, 98596, 99453, 99542, 99593, 101831, 102183, 102416, 103107, 103108, 105677, 106041]

In [ ]:
df.drop(rem, axis = 0, inplace=True)
df.reset_index(inplace=True, drop=True )

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106158 entries, 0 to 106157
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Tweet ID  106158 non-null  object
 1   Tweets    106158 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


###3. Removing usernames/mentions, emails, links and tags(they do not provide any useful information for the classification process)


In [ ]:
import re
for i in range(len(df)):
  Tweet = df['Tweets'].iloc[i]
  Tweet = re.sub('@[^\s]+',' ',Tweet) #usernames
  Tweet = re.sub('[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+',' ',Tweet) #email IDs
  Tweet = re.sub(r"http\S+", " ", Tweet) #links
  Tweet = re.sub(r"#[A-Za-z0-9]+", " ", Tweet) #tags
  df.at[i,'Tweets'] = Tweet


###4. Extracting hashtags into a different array so that i can remove them from original tweet

In [ ]:
import re
hashtags = []
for i in range(len(df)):
  Tweet = df['Tweets'].iloc[i]
  hashtag = re.findall(r"#(\w+)", Tweet)
  hashtags.append(hashtag)
  Tweet = re.sub('\n', ' ', Tweet)
  Tweet = re.sub('\r', ' ', Tweet)
  Tweet = re.sub('\t', ' ', Tweet)
  words = Tweet.split()
  table = str.maketrans(' ', ' ', '.!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~—‘’”“')
  stripped = [w.translate(table) for w in words]
  df.at[i,'Tweets'] = " ".join(stripped)


In [ ]:
hashtags[0:5]

[]

###5. Lowercasing

In [ ]:
for i in range(len(df)):
    Tweet = df['Tweets'].iloc[i]
    df.at[i,'Tweets'] = Tweet.lower()

###6. Replacing emoticons with corresponding words 

In [ ]:
!pip install emot

In [ ]:
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

def convert_emoticons(text):
  for emot in EMOTICONS:
    text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
  return text

def convert_emojis(text):
  for emot in UNICODE_EMO:
    s = " ".join(UNICODE_EMO[emot].replace(",","").replace(":","").split("_"))
    s = " "+s+" "
    text = text.replace(emot, s)
  return text
  

'Hello Happy face or smiley Happy face smiley'

In [ ]:
for i in range(len(df)):
  Tweet = df['Tweets'].iloc[i]
  Tweet = convert_emojis(Tweet)
  df.at[i,'Tweets'] = Tweet
  Tweet = convert_emoticons(Tweet)
  df.at[i,'Tweets'] = Tweet

###7. Stop word removal

In [ ]:
import nltk
nltk.download("stopwords") 
nltk.download('punkt')
from nltk.corpus import stopwords
stopword = stopwords.words('english')
for i in range(len(df)):
  Tweet = df['Tweets'].iloc[i]
  word_tokens = nltk.word_tokenize(Tweet)
  removing_stopwords = [word for word in word_tokens if word not in stopword]
  df.at[i,'Tweets'] = " ".join(removing_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### I noticed that after these steps some duplicates showed up, which were not identified earlier due to casing issues or maybe some punctuation. So i remove the duplicates here

In [ ]:
df.drop_duplicates(subset = 'Tweets', keep = 'first', inplace = True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84300 entries, 0 to 84299
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tweet ID  84300 non-null  object
 1   Tweets    84300 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [ ]:
df.sample(7)

,Tweet ID,Tweets
25086,1386043303687303173,lockdown lagao or end up like india or lose ou...
4994,1383909838955056896,mumbai on saturday recorded 8811 new cases and...
29707,1386303181429219335,anytimefitness kempscorner dont let lockdown a...
67110,1389537390465753092,consider imposing lockdown to control second w...
7223,1383992515573871104,okay if we write the exams and if any student ...
24535,1385885439685382151,extend delhi covid19 lockdown by another week ...
71949,1389911035444965380,government of india faces coronovirus lockdown...
